## Carregando bibliotecas 

In [6]:
### Import required libraries

import numpy as np
import pandas as pd
import gc

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

from IPython.display import display # Allows the use of display() for DataFrames

import warnings
warnings.filterwarnings('ignore')

## Carregando dados de teste e treino

In [7]:
# Read train and test files
treino = pd.read_csv('../input/train.csv')
teste = pd.read_csv('../input/test.csv')

### Dados de Treino

In [8]:
treino.head()

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,11d86fa6a,77c9823f2,8d6c2a0b2,4681de4fd,adf119b9a,cff75dd09,96f83a237,b8a716ebf,6c7a4567c,4fcfd2b4d,f3b9c0b95,71cebf11c,d966ac62c,68b647452,c88d108c9,ff7b471cd,d5308d8bc,0d866c3d7,bc3f77679,bd8f989f1,0eff5bf95,22ed6dba3,92b13ebba,c330f1a67,233c7c17c,2cb4d123e,eeac16933,87ffda550,...,969caa87a,00302fe51,1189ee335,ca04a07ca,f6f15ffa5,841704460,ea5ed6ff7,b1bb8eac3,8132d18b8,c24ea6548,cdfc2b069,2a879b4f7,6b119d8ce,98dea9e42,9f2471031,88458cb21,f40da20f4,7ad6b38bd,c901e7df1,8f55955dc,85dcc913d,5ca0b9b0c,eab8abf7a,8d8bffbae,2a1f6c7f9,9437d8b64,5831f4c76,2e84e09c5,d45fd5508,a165f5761,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000.0,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,1300000.0,...,0,0,1100000.0,0,0,0,0.0,0.0,14800000,0.0,1200000.0,0.0,0.0,0,0,0,0,0.0,4000000,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
1,000fbd867,600000.0,0.0,0,0.0,0,0,0,0,0,2200000.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0.0,...,0,0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0027d6b71,10000000.0,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0.0,...,0,0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0028cbf45,2000000.0,0.0,0,0.0,0,0,0,0,0,0.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0.0,...,0,0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0
4,002a68644,14400000.0,0.0,0,0.0,0,0,0,0,0,2000000.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0,0.0,...,0,0,0.0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0


### Dados de teste

In [9]:
teste.head()

,ID,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,11d86fa6a,77c9823f2,8d6c2a0b2,4681de4fd,adf119b9a,cff75dd09,96f83a237,b8a716ebf,6c7a4567c,4fcfd2b4d,f3b9c0b95,71cebf11c,d966ac62c,68b647452,c88d108c9,ff7b471cd,d5308d8bc,0d866c3d7,bc3f77679,bd8f989f1,0eff5bf95,22ed6dba3,92b13ebba,c330f1a67,233c7c17c,2cb4d123e,eeac16933,87ffda550,822e49b95,...,969caa87a,00302fe51,1189ee335,ca04a07ca,f6f15ffa5,841704460,ea5ed6ff7,b1bb8eac3,8132d18b8,c24ea6548,cdfc2b069,2a879b4f7,6b119d8ce,98dea9e42,9f2471031,88458cb21,f40da20f4,7ad6b38bd,c901e7df1,8f55955dc,85dcc913d,5ca0b9b0c,eab8abf7a,8d8bffbae,2a1f6c7f9,9437d8b64,5831f4c76,2e84e09c5,d45fd5508,a165f5761,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000137c73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.528249e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00021489f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0004d7953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00056a333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,20884000.0,0.0,0.0,0.0,0.0,0.0,36252000.0,0.0,20000000.0,0.0,...,0.0,0.0,25010000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00056d8eb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1ª Percepção sobre os dados

In [10]:
treino.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: float64(1845), int64(3147), object(1)
memory usage: 169.9+ MB


In [11]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49342 entries, 0 to 49341
Columns: 4992 entries, ID to 9fc776466
dtypes: float64(4991), object(1)
memory usage: 1.8+ GB


### Para os dados de treino temos que:

Um total de 4993 colunas, das quais 1845 são do tipo float64, 3147 são int64 e 1 é objeto (o ID é a coluna de objeto). object (ID is the object column)

### Para os dados de teste temos que:

Um total de 4992 colunas no conjunto de teste, das quais 4991 são do tipo float64 e 1 é objeto (o ID é a coluna de objeto).

### O que se conclui com essas métricas das bases de dados:

Em grandes bases de dados, é comum encontrar um grande volume de features, mas nem todas são relevantes para a tarefa em questão. Algumas podem ser redundantes, ter baixa variabilidade ou não contribuir significativamente para a previsão ou classificação. A remoção de features desnecessárias, como as constantes ou com baixa correlação com a variável-alvo, é essencial para melhorar o desempenho do modelo. Isso reduz a dimensionalidade do dataset, acelera o tempo de processamento e diminui o risco de overfitting, além de facilitar a interpretação dos resultados.

Algoritmos como o **LightGBM** são especialmente úteis para lidar com grandes bases de dados. O LightGBM, por exemplo, é uma biblioteca de aprendizado de máquina baseada em **gradiente boosting**, otimizada para grandes volumes de dados e alta performance. Ele é capaz de lidar com dados grandes de forma eficiente devido ao seu uso de histogramas, que aceleram o processo de treinamento e reduzem o uso de memória. Além disso, o LightGBM é projetado para lidar com features categóricas e suporta paralelização, o que o torna uma escolha poderosa para cenários de grande escala.

Assim, a combinação da remoção de features irrelevantes e o uso de algoritmos eficientes como LightGBM resulta em modelos mais rápidos e precisos, mesmo em bases de dados complexas e massivas.

## Verificar se existem valores NaN

In [13]:
print("Total de Recursos de Treinamento com Valores NaN = " + str(treino.columns[treino.isnull().sum() != 0].size))
if (treino.columns[treino.isnull().sum() != 0].size):
    print("Recursos com NaN => {}".format(list(treino.columns[treino.isnull().sum() != 0])))
    treino[treino.columns[treino.isnull().sum() != 0]].isnull().sum().sort_values(ascending = False)

Total de Recursos de Treinamento com Valores NaN = 0


In [14]:
print("Total de Recursos de Teste com Valores NaN = " + str(teste.columns[teste.isnull().sum() != 0].size))
if (teste.columns[teste.isnull().sum() != 0].size):
    print("Recursos com NaN => {}".format(list(teste.columns[teste.isnull().sum() != 0])))
    teste[teste.columns[teste.isnull().sum() != 0]].isnull().sum().sort_values(ascending = False)

Total de Recursos de Teste com Valores NaN = 0


## Removendo FEATURES constantes

Bom salientar que a remoção de features constantes em uma grande base de dados é importante porque essas variáveis não trazem informações relevantes para o modelo. Como elas possuem o mesmo valor em todas as observações, não contribuem para distinguir ou explicar padrões nos dados. Além disso, manter features constantes aumenta a complexidade do modelo desnecessariamente, o que pode impactar o desempenho e o tempo de processamento. Portanto, removê-las otimiza o processo de análise, tornando o modelo mais eficiente e preciso.

In [15]:
# check and remove constant columns
colsToRemove = []
for col in treino.columns:
    if col != 'ID' and col != 'target':
        if treino[col].std() == 0: 
            colsToRemove.append(col)
        
# remove constant columns in the training set
treino.drop(colsToRemove, axis=1, inplace=True)

# remove constant columns in the test set
teste.drop(colsToRemove, axis=1, inplace=True) 

print("Foram removidas `{}` Constant Columns\n".format(len(colsToRemove)))
print(colsToRemove)

Foram removidas `256` Constant Columns

['d5308d8bc', 'c330f1a67', 'eeac16933', '7df8788e8', '5b91580ee', '6f29fbbc7', '46dafc868', 'ae41a98b6', 'f416800e9', '6d07828ca', '7ac332a1d', '70ee7950a', '833b35a7c', '2f9969eab', '8b1372217', '68322788b', '2288ac1a6', 'dc7f76962', '467044c26', '39ebfbfd9', '9a5ff8c23', 'f6fac27c8', '664e2800e', 'ae28689a2', 'd87dcac58', '4065efbb6', 'f944d9d43', 'c2c4491d5', 'a4346e2e2', '1af366d4f', 'cfff5b7c8', 'da215e99e', '5acd26139', '9be9c6cef', '1210d0271', '21b0a54cb', 'da35e792b', '754c502dd', '0b346adbd', '0f196b049', 'b603ed95d', '2a50e001c', '1e81432e7', '10350ea43', '3c7c7e24c', '7585fce2a', '64d036163', 'f25d9935c', 'd98484125', '95c85e227', '9a5273600', '746cdb817', '6377a6293', '7d944fb0c', '87eb21c50', '5ea313a8c', '0987a65a1', '2fb7c2443', 'f5dde409b', '1ae50d4c3', '2b21cd7d8', '0db8a9272', '804d8b55b', '76f135fa6', '7d7182143', 'f88e61ae6', '378ed28e0', 'ca4ba131e', '1352ddae5', '2b601ad67', '6e42ff7c7', '22196a84c', '0e410eb3d', '992e6d1d3

## Removendo colunas duplicadas

In [16]:
%%time
def duplicate_columns(frame):
    groups = frame.columns.to_series().groupby(frame.dtypes).groups
    dups = []

    for t, v in groups.items():

        cs = frame[v].columns
        vs = frame[v]
        lcs = len(cs)

        for i in range(lcs):
            ia = vs.iloc[:,i].values
            for j in range(i+1, lcs):
                ja = vs.iloc[:,j].values
                if np.array_equal(ia, ja):
                    dups.append(cs[i])
                    break

    return dups

colsToRemove = duplicate_columns(treino)
print(colsToRemove)

['34ceb0081', '8d57e2749', '168b3e5bc', 'a765da8bc', 'acc5b709d']
CPU times: user 10min 46s, sys: 105 ms, total: 10min 46s
Wall time: 10min 46s


In [17]:
# remove duplicate columns in the training set
treino.drop(colsToRemove, axis=1, inplace=True) 

# remove duplicate columns in the testing set
teste.drop(colsToRemove, axis=1, inplace=True)

print("Removed `{}` Duplicate Columns\n".format(len(colsToRemove)))
print(colsToRemove)

Removed `5` Duplicate Columns

['34ceb0081', '8d57e2749', '168b3e5bc', 'a765da8bc', 'acc5b709d']


## Removendo colunas de baixa variabilidade/colunas esparsas

Remover colunas com baixa variabilidade ou esparsas é essencial para melhorar a performance dos modelos de machine learning. Colunas com valores quase sempre iguais ou muitos nulos/zeros não contribuem com informações úteis, adicionam **ruído** e podem levar ao **overfitting**. Além disso, manter essas colunas aumenta o tempo de processamento e o uso de memória desnecessariamente. Ao eliminá-las, o modelo se torna mais eficiente, focando nas features que realmente importam, resultando em melhores previsões e menor custo computacional.

In [18]:
def drop_sparse(train, test):
    flist = [x for x in train.columns if not x in ['ID','target']]
    for f in flist:
        if len(np.unique(train[f]))<2:
            train.drop(f, axis=1, inplace=True)
            test.drop(f, axis=1, inplace=True)
    return train, test

In [19]:
%%time
treino, teste = drop_sparse(treino, teste)

CPU times: user 788 ms, sys: 5 µs, total: 788 ms
Wall time: 787 ms


In [20]:
gc.collect()
print("Train set size: {}".format(treino.shape))
print("Test set size: {}".format(teste.shape))

Train set size: (4459, 4732)
Test set size: (49342, 4731)


## Criando o Split de treino e teste

In [21]:
X_train = treino.drop(["ID", "target"], axis=1)
y_train = np.log1p(treino["target"].values)

X_test = teste.drop(["ID"], axis=1)

In [22]:
dev_X, val_X, dev_y, val_y = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)

## Aplicando o [LightGBM](http://lightgbm.readthedocs.io/en/stable/index.html)

In [23]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        "num_leaves" : 40,
        "learning_rate" : 0.004,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_frequency" : 6,
        "bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 5000, 
                      valid_sets=[lgtrain, lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=150, 
                      evals_result=evals_result)
    
    pred_test_y = np.expm1(model.predict(test_X, num_iteration=model.best_iteration))
    return pred_test_y, model, evals_result

In [24]:
# Training LGB
pred_test, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, X_test)
print("LightGBM Training Completed...")

Training until validation scores don't improve for 100 rounds.
[150]	training's rmse: 1.50793	valid_1's rmse: 1.53931
[300]	training's rmse: 1.34402	valid_1's rmse: 1.46602
[450]	training's rmse: 1.23291	valid_1's rmse: 1.43402
[600]	training's rmse: 1.14927	valid_1's rmse: 1.41918
[750]	training's rmse: 1.08343	valid_1's rmse: 1.41347
[900]	training's rmse: 1.03011	valid_1's rmse: 1.41164
[1050]	training's rmse: 0.985188	valid_1's rmse: 1.4116
Early stopping, best iteration is:
[976]	training's rmse: 1.00631	valid_1's rmse: 1.41125
LightGBM Training Completed...


In [25]:
# feature importance
print("Features Importance...")
gain = model.feature_importance('gain')
featureimp = pd.DataFrame({'feature':model.feature_name(), 
                   'split':model.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(featureimp[:50])

Features Importance...
        feature  split      gain
4130  f190486d6    803  9.017404
2375  58e2e02e6    715  5.294145
3465  eeb9cd3aa    675  4.294044
4020  15ace8c9f    541  3.095043
2614  9fd594eec    379  2.850291
8     20aa07010    409  2.112768
834   6eef030c1    342  1.400379
3571  58232a6fb    380  1.400253
1457  b43a7cfd5    399  1.187425
3661  491b9ee45    297  1.048269
2687  fb0f5dbfe    410  0.997311
1482  024c577b9    258  0.931900
3867  2288333b4    195  0.860454
2079  58e056e12    347  0.857672
4185  f74e8f13d    352  0.849962
4343  1702b5bf0    291  0.836605
4508  c47340d97    313  0.815153
828   6786ea46d    184  0.798005
566   66ace2992    276  0.778013
3722  d6bb78916    313  0.770098
3791  ed8ff54b5    169  0.764647
3220  ced6a7e91    240  0.720281
4028  5c6487af1    180  0.712760
3886  50e4f96cf    152  0.686449
863   fc99f9426    238  0.667129
1378  6cf7866c1    160  0.621800
34    87ffda550    162  0.604544
853   bc70cbc26    139  0.599770
3811  adb64ff71    2

## XGB Modeling

In [26]:
def run_xgb(train_X, train_y, val_X, val_y, test_X):
    params = {'objective': 'reg:linear', 
          'eval_metric': 'rmse',
          'eta': 0.001,
          'max_depth': 10, 
          'subsample': 0.6, 
          'colsample_bytree': 0.6,
          'alpha':0.001,
          'random_state': 42, 
          'silent': True}
    
    tr_data = xgb.DMatrix(train_X, train_y)
    va_data = xgb.DMatrix(val_X, val_y)
    
    watchlist = [(tr_data, 'train'), (va_data, 'valid')]
    
    model_xgb = xgb.train(params, tr_data, 2000, watchlist, maximize=False, early_stopping_rounds = 100, verbose_eval=100)
    
    dtest = xgb.DMatrix(test_X)
    xgb_pred_y = np.expm1(model_xgb.predict(dtest, ntree_limit=model_xgb.best_ntree_limit))
    
    return xgb_pred_y, model_xgb

In [27]:
# Training XGB
pred_test_xgb, model_xgb = run_xgb(dev_X, dev_y, val_X, val_y, X_test)
print("XGB Training Completed...")

[0]	train-rmse:14.0877	valid-rmse:14.0768
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[100]	train-rmse:12.7683	valid-rmse:12.756
[200]	train-rmse:11.5765	valid-rmse:11.5629
[300]	train-rmse:10.4999	valid-rmse:10.4854
[400]	train-rmse:9.52803	valid-rmse:9.51304
[500]	train-rmse:8.65065	valid-rmse:8.6361
[600]	train-rmse:7.85865	valid-rmse:7.84473
[700]	train-rmse:7.14388	valid-rmse:7.13201
[800]	train-rmse:6.49911	valid-rmse:6.48993
[900]	train-rmse:5.91691	valid-rmse:5.91106
[1000]	train-rmse:5.3925	valid-rmse:5.39137
[1100]	train-rmse:4.9196	valid-rmse:4.92423
[1200]	train-rmse:4.49343	valid-rmse:4.50457
[1300]	train-rmse:4.10977	valid-rmse:4.12874
[1400]	train-rmse:3.76474	valid-rmse:3.79266
[1500]	train-rmse:3.45493	valid-rmse:3.49292
[1600]	train-rmse:3.17654	valid-rmse:3.22512
[1700]	train-rmse:2.92698	valid-rmse:2.98712
[1800]	train-rmse:2.70352	valid-rmse:2.77675
[1900]	train-rm

## Conclusão

O LightGBM apresentou uma performance superior em termos de RMSE, com um erro final de 1.41125, contra 2.42818 do XGBoost, sugerindo que ele conseguiu ajustar-se melhor ao conjunto de dados com menos iterações. Além disso, o LightGBM atingiu o early stopping em 976 iterações, enquanto o XGBoost continuou treinando até 1999 iterações, indicando que o LightGBM foi mais eficiente, tanto em tempo de treinamento quanto em performance final. Portanto, para esse cenário específico, o LightGBM oferece melhor precisão e convergência mais rápida.


A diferença na performance entre XGBoost e LightGBM pode ser explicada pelas diferenças na forma como cada algoritmo constrói suas árvores e otimiza o processo de aprendizado. O LightGBM, com seu crescimento baseado em folhas, uso eficiente de histogramas e paralelização otimizada, consegue convergir mais rápido e com menos iterações, o que justifica o menor RMSE observado no seu experimento. Já o XGBoost, com uma abordagem mais conservadora e tradicional, é mais lento para ajustar o modelo e necessita de mais iterações para alcançar um resultado competitivo.